## Python笔记：关于 funcion 和 bound method

> https://stackoverflow.com/questions/7891277/why-does-setattr-fail-on-a-bound-method


### 遇到的问题：试图对成员函数setattr时会抛出错误

AttributeError: 'method' object has no attribute 'foo'

### 原理：function 与 bound method 的本质是什么？

function object 是某种“真实存在的”对象，可以设置任意attr，它有一个`__dict__`属性来存储所有的attr    

而与特定实例绑定的 bound method 不同，它没有具体的函数实体。

class中的def会创建一个（属于cls命名空间中的）funcion object，通过`cls.m`获得。   
而与具体实例绑定的 bound method 不存储函数实体，它只是是记录两个对象的引用：

- 其宿主 `instance.__self__`
- 定义在类中的函数实体 `instance.m.__func__` (is `cls.m`)

当bound method 被调用时，实际执行的是`cls.m(instance.__self__, ...)`   
该类所有实例的method，其函数实体`instance.m.__func__` 都指向同一个对象，即类创建时创建的 function object `cls.m`

> 回忆quantx是怎么在R语言中实现“类”的？

### 回到刚才的问题

commit e785ee8 实际上是将属性设置在了cls.m上（定义在class namespace中，是函数对象），是错误的。

```
def dec(func):
    def wrapper(*arg, **kw):
        func.xx = xx              # ✔️ 不会报错，但注意，属性是设置在了 KLS.func上
        return func(*arg, **kw)
    return wrapper

class KLS:
    @dec
    def foo:
        setattr(self.bar, 'xx', xx)   # ❌ bounded method 不是函数对象，不可以设置属性
        setattr(self.foo, 'xx, xx)  # ❌原因同上

ins = KLS()
ins.foo.xx = xx   # ❌ 原因同上
```

In [26]:
m_bb.swap is m_bb2.swap   # different object, since it is bound method of two different istance

False

In [24]:
m_bb.swap.__func__ is m_bb2.swap.__func__  # same object ...

True

In [29]:
m_bb.swap.__func__ is BubbleSort.swap    # ... since they are all ref to cls.funcion

True

In [31]:
m_bb.swap is BubbleSort.swap   # bound method and cls.funcion is not same thing !

False

In [18]:
BubbleSort.swap  # it is function object !

<function __main__.Sorting.swap(self, L, i, j)>

In [19]:
m_bb.swap  # it is bound method

<bound method Sorting.swap of <__main__.BubbleSort object at 0x1057068d0>>

In [20]:
m_bb2.swap  # bound method of another istance

<bound method Sorting.swap of <__main__.BubbleSort object at 0x105706390>>

### 那么怎么实现“成员方法调用次数计数”这个需求呢？我自己想的一个实现：decorator最原始的语法

> https://medium.com/@vadimpushtaev/decorator-inside-python-class-1e74d23107f6

1. 直接在成员方法的`def`上加`@dec`并不work
2. staticmethod也不行，因为 @staticmethod 对于inside class / outside class 的behavior不同

In [63]:
class A(object):
    def __init__(self):
        print('calling __init__')
        self.count_call_dict = {}
        
        self.a = self.count_call(self.a)
        self.b = self.count_call(self.b)
        
    def a(self):
        print('aaaaaaaaaaa')
    
    def b(self):
        print('bbbbbbbbbbb')
        
    def count_call(self, func):
        print('Calling count_call')
        self.count_call_dict[func.__name__] = 0
        def wrapper(*args, **kw):
            print('func name is', func.__name__)
            self.count_call_dict[func.__name__] += 1
            return func(*args, **kw)
        return wrapper

In [69]:
foooo = A()
b___ = A()

foooo.a(); foooo.a(); foooo.a()   # foooo.a = 3
foooo.b(); foooo.b()              # foooo.b = 2
b___.a()
foooo.a()                         # foooo.a = 4
b___.b(); b___.b()

print(foooo.count_call_dict)
print(b___.count_call_dict)

calling __init__
Calling count_call
Calling count_call
calling __init__
Calling count_call
Calling count_call
func name is a
aaaaaaaaaaa
func name is a
aaaaaaaaaaa
func name is a
aaaaaaaaaaa
func name is b
bbbbbbbbbbb
func name is b
bbbbbbbbbbb
func name is a
aaaaaaaaaaa
func name is a
aaaaaaaaaaa
func name is b
bbbbbbbbbbb
func name is b
bbbbbbbbbbb
{'a': 4, 'b': 2}
{'a': 1, 'b': 2}


## Python Notes: pass by assignment

> [python document](https://docs.python.org/3/faq/programming.html#how-do-i-write-a-function-with-output-parameters-call-by-reference)    
> "Arguments are **passed by assignment** in Python."

> [A blog](https://robertheaton.com/2014/02/09/pythons-pass-by-object-reference-as-explained-by-philip-k-dick/)    
> What does it means by saying **"Object references are passed by value"**?

> https://nedbatchelder.com/text/names1.html


In [3]:
# 下面的例子说明，python没有所谓的 “传引用”

a = [0, 1, 2]
b = a[1]   # 不是引用 b = &a[1]，而是 name b bounded to object in a[1]  想象一下示意图

b = 99
a          # 如果是 b = &a[1]，那么 a 应该为 [0, 99, 2]

[0, 1, 2]